# Install protinfer 

```
git clone https://github.com/google-research/proteinfer
cd ~/proteinfer
pip3 install -r requirements.txt
python3 install_models.py
```

Example to run (you don't need this for our benchmarking pipeline but it may be useful to know!)

```
python3 proteinfer.py -i /disk1/ariane/pycharm/CARE/CLEAN/app/data/price.fasta -o ~/hemoglobin_predictions.tsv
```


## Run protinfer

Note protinfer was not re-trained so may be slightly biased.

Protinfer needs to be installed as per the guidelines and then the following command can be run from the protinfer folder:

## Run CARE

In [6]:
import sys
sys.path.append('/disk1/ariane/vscode/CARE/')
sys.path.append('/disk1/ariane/vscode/CARE/CARE/')

from CARE.processing import *
from CARE.task1 import *

data_dir = '/disk1/ariane/vscode/CARE/data/task1/'
output_dir = '/disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/'

In [8]:
# Save in the required format
# Make a task1
tasker = Task1(data_folder=data_dir, output_folder=output_dir)
for split in ['30', '30-50', 'price', 'promiscuous']:
    # For proteInfer you need to point where it was saved.
    tasker.get_proteinfer(split, proteinfer_dir='/disk1/ariane/pycharm/CARE/proteinfer/', save=True)

--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/30_protein_test.fasta





I0604 09:36:27.174640 140249102206784 proteinfer.py:392] Running with 1 ensemble elements
I0604 09:36:27.174786 140249102206784 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/30_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]

Annotating batches of sequences: 100%|██████████| 3/3 [00:06<00:00,  2.11s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.80s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:04<00:00,  1.66s/it]]

Progress: 100%|██████████| 3/3 [00:16<00:00,  5.64s/it]
I0604 09:36:46.485210 140249102206784 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30_proteInfer.tsv



--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/30-50_protein_test.fasta





I0604 09:36:49.547793 140079762843456 proteinfer.py:392] Running with 1 ensemble elements
I0604 09:36:49.547944 140079762843456 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/30-50_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.75s/it]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]

Annotating batches of sequences: 100%|██████████| 4/4 [00:06<00:00,  1.52s/it]

Progress: 100%|██████████| 3/3 [00:19<00:00,  6.65s/it]
I0604 09:37:11.890649 140079762843456 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/30-50_proteInfer.tsv



--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/price_protein_test.fasta



--------------------------------------------------------------------------------
        Warning! You must already have the proteInfer environment made	         
--------------------------------------------------------------------------------
/disk1/ariane/vscode/CARE/data/task1/promiscuous_protein_test.fasta


I0604 09:37:14.941128 140267527726912 proteinfer.py:392] Running with 1 ensemble elements
I0604 09:37:14.941279 140267527726912 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/price_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.30it/s]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.73s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:04<00:00,  1.60s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:05<00:00,  1.71s/it]]

Progress: 100%|██████████| 3/3 [00:15<00:00,  5.11s/it]
I0604 09:37:32.657966 140267527726912 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/price_proteInfer.tsv



I0604 09:37:35.007777 139752283248448 proteinfer.py:392] Running with 1 ensemble elements
I0604 09:37:35.007924 139752283248448 proteinfer.py:189] Parsing input from /disk1/ariane/vscode/CARE/data/task1/promiscuous_protein_test.fasta
Loading models: 100%|██████████| 3/3 [00:02<00:00,  1.30it/s]

Annotating batches of sequences: 100%|██████████| 3/3 [00:06<00:00,  2.31s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:06<00:00,  2.29s/it]

Annotating batches of sequences: 100%|██████████| 3/3 [00:06<00:00,  2.21s/it]]

Progress: 100%|██████████| 3/3 [00:20<00:00,  6.88s/it]
I0604 09:37:58.109966 139752283248448 proteinfer.py:340] Writing output to /disk1/ariane/vscode/CARE/task1_baselines/results_summary/ProteInfer/promiscuous_proteInfer.tsv



## Train the model
Future work may include training the model:
```
python train.py --data_base_path=./testdata/ \
--label_vocab_path=./data/vocabs/EC.tsv \
--hparams_set=small_test_model \
--output_dir=trained_baseline1
```